<a href="https://colab.research.google.com/github/santanukumar666/data_analysis_projects/blob/main/Copy_of_Zomato_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### so that u dont have warnings
from warnings import filterwarnings
filterwarnings('ignore')

#### Columns description

1. url contains the url of the restaurant in the zomato website

2. address contains the address of the restaurant in Bengaluru

3. name contains the name of the restaurant

4. online_order whether online ordering is available in the restaurant or not

5. book_table table book option available or not

6. rate contains the overall rating of the restaurant out of 5

7. votes contains total number of rating for the restaurant as of the above mentioned date

8. phone contains the phone number of the restaurant

9. location contains the neighborhood in which the restaurant is located

10. rest_type restaurant type

11. dish_liked dishes people liked in the restaurant

12. cuisines food styles, separated by comma

13. approx_cost(for two people) contains the approximate cost for meal for two people

14. reviews_list list of tuples containing reviews for the restaurant, each tuple

15. menu_item contains list of menus available in the restaurant

16. listed_in(type) type of meal

17. listed_in(city) contains the neighborhood in which the restaurant is listed

In [ ]:
#read dataset
df=pd.read_csv(r'C:\Users\subra\zomato_data_analysis/zomato.csv')
df.head()

FileNotFoundError: ignored

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
len(df['name'].unique())

In [ ]:
df.isna().sum()

#### getting all NAN features

In [ ]:
feature_na=[feature for feature in df.columns if df[feature].isnull().sum()>0]
feature_na

In [ ]:
#% of missing values
for feature in feature_na:
    print('{} has {} % missing values'.format(feature,np.round(df[feature].isnull().sum()/len(df)*100,4)))

In [ ]:
df['rate'].unique()

In [ ]:
df.dropna(axis='index',subset=['rate'],inplace=True)

In [ ]:
df.shape

In [ ]:
def split(x):
    return x.split('/')[0]

In [ ]:
df['rate']=df['rate'].apply(split)

In [ ]:
df.head()

In [ ]:
df['rate'].unique()

In [ ]:
df.replace('NEW',0,inplace=True)

In [ ]:
df.replace('-',0,inplace=True)

In [ ]:
df['rate']=df['rate'].astype(float)

### calculate avg rating of each resturant

In [ ]:
df.groupby('name')['rate'].mean().nlargest(20).plot.bar()

In [ ]:
df_rate=df.groupby('name')['rate'].mean().to_frame()
df_rate=df_rate.reset_index()
df_rate.columns=['restaurant','rating']
df_rate.head(20)

In [ ]:
df_rate.shape

#### alternative is create a list in which u have all the restaurants & in another list,we have all ratings & then using zip we can create a dataframe

In [ ]:
'''restaurant=[]
avg_rating=[]
for key,name_df in df.groupby('name'):
    restaurant.append(key)
    avg_rating.append(np.mean(name_df['rate'])'''

In [ ]:
'''df_rate=pd.DataFrame(zip(restaurant,avg_rating))
df_rate.columns=['restaurant','rating']
df_rate.head(20)'''

In [ ]:
'''df_rate.shape'''

##### Rating distribution

In [ ]:
sns.set_style(style='whitegrid')
sns.distplot(df_rate['rating'])

##### Almost more than 50 percent of restaurants has rating between 3 and 4. Restaurants having rating more than 4.5 are very rare.

#### Which are the top restaurant chains in Bangaluru?

In [ ]:
plt.figure(figsize=(10,7))
chains=df['name'].value_counts()[0:20]
sns.barplot(x=chains,y=chains.index,palette='deep')
plt.title("Most famous restaurants chains in Bangaluru")
plt.xlabel("Number of outlets") 

##### How many of the restuarants do not accept online orders?

In [ ]:
x=df['online_order'].value_counts()
labels=['accepted','not accepted']
plt.pie(x,explode=[0.0,0.1],autopct='%1.1f%%')

##### using plotly, How many of the restuarants do not accept online orders?

In [ ]:
!pip install plotly

In [ ]:
import plotly.express as px

In [ ]:
x=df['online_order'].value_counts()
labels=['accepted','not accepted']

In [ ]:
fig = px.pie(df, values=x, names=labels,title='Pie chart')
fig.show()

#### What is the ratio b/w restaurants that provide and do not provide table booking ?

In [ ]:
x=df['book_table'].value_counts()
labels=['not book','book']
plt.pie(x,explode=[0.0,0.1],autopct='%1.1f%%')

##### using plotly, #### What is the ratio b/w restaurants that provide and do not provide table booking ?

In [ ]:
import plotly.graph_objs as go
from plotly.offline import iplot

In [ ]:
x=df['book_table'].value_counts()
labels=['not book','book']

In [ ]:
trace=go.Pie(labels=labels, values=x,
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=25),
              pull=[0, 0, 0,0.2, 0]
               )

In [ ]:
iplot([trace])

#### How many types of restaurants we have?

In [ ]:
df['rest_type'].isna().sum()

In [ ]:
df['rest_type'].dropna(inplace=True)

In [ ]:
df['rest_type'].isna().sum()

In [ ]:
len(df['rest_type'].unique())

In [ ]:
plt.figure(figsize=(20,12))
df['rest_type'].value_counts().nlargest(20).plot.bar(color='red')
plt.gcf().autofmt_xdate()

##### now using plotly

In [ ]:
trace1 = go.Bar( 
        x = df['rest_type'].value_counts().nlargest(20).index,
        y = df['rest_type'].value_counts().nlargest(20),
        name= 'rest_type')

In [ ]:
iplot([trace1])

##### At all, Banglore is known as the tech capital of India,people having busy and modern life will prefer Quick Bites.
##### We can observe tha Quick Bites type restaurants dominates.

#### highest voted restaurant

In [ ]:
df.groupby('name')['votes'].max().nlargest(10).plot.bar()
    

##### now using plotly to  provide interactive graphs

In [ ]:
trace1 = go.Bar( 
        x = df.groupby('name')['votes'].max().nlargest(10).index,
        y = df.groupby('name')['votes'].max().nlargest(10),
        name= 'name')

In [ ]:
iplot([trace1])

### total restaurants at different locations of Bengalore

In [ ]:
df.groupby('location')['name'].unique()

In [ ]:
restaurant=[]
location=[]
for key,location_df in df.groupby('location'):
    location.append(key)
    restaurant.append(len(location_df['name'].unique()))


In [ ]:
df_total=pd.DataFrame(zip(location,restaurant))
df_total.columns=['location','restaurant']
df_total.set_index('location',inplace=True)
df_total.sort_values(by='restaurant').tail(10)

In [ ]:
df_total.sort_values(by='restaurant').tail(10).plot.bar()


In [ ]:
df_total.sort_values(by='restaurant').tail(10).index

In [ ]:
trace1 = go.Bar( 
        x = df_total['restaurant'].nlargest(10).index,
        y = df_total['restaurant'].nlargest(10),
        name= 'Priority')

In [ ]:
iplot([trace1])

In [ ]:
df.isnull().sum()

### different type of restaurants

In [ ]:
#data is represented in form of percentage 
(df['rest_type'].value_counts()/len(df))*100

### Total number of variety of restaurants ie north indian,south Indian

In [ ]:

cuisines=df['cuisines'].value_counts()[:10]
sns.barplot(cuisines,cuisines.index)
plt.xlabel('Count')
plt.title("Most popular cuisines of Bangalore")

In [ ]:
cuisines=df['cuisines'].value_counts()[:10]
trace1 = go.Bar( 
        x = cuisines.index,
        y = cuisines,
        name= 'Cuisines')

In [ ]:
iplot([trace1])

### We can observe that North Indian,chinese,South Indian and Biriyani are most common.
### It means Bengalore is more influenced by North Indian culture more than South

### Analyse Approx cost for 2 people

In [ ]:
len(df['approx_cost(for two people)'].value_counts())

NameError: ignored

In [ ]:
df['approx_cost(for two people)'].isna().sum()

In [ ]:
df.dropna(axis='index',subset=['approx_cost(for two people)'],inplace=True)

In [ ]:
df['approx_cost(for two people)'].isna().sum()

In [ ]:
df['approx_cost(for two people)'].unique()

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
df[~df['approx_cost(for two people)'].apply(is_float)]

In [ ]:
df['approx_cost(for two people)'].dtype

In [ ]:
df['approx_cost(for two people)']

In [ ]:
type(df['approx_cost(for two people)'][0])

#### remove ,

In [ ]:
df['approx_cost(for two people)'] = df['approx_cost(for two people)'].apply(lambda x: x.replace(',',''))

In [ ]:
df['approx_cost(for two people)']=df['approx_cost(for two people)'].astype(int)

##### cost vs rating

In [ ]:
plt.figure(figsize=(10,7))
sns.scatterplot(x="rate",y='approx_cost(for two people)',hue='online_order',data=df)
plt.show()

##### from this scattterplot,we can come up with a conclusion ya most of the highest rated accepts online order and they r budgeted too

In [ ]:
df.head()

### Is there any difference b/w votes of restaurants accepting and not accepting online orders?

In [ ]:
sns.boxplot(x='online_order',y='votes',data=df)

#### from this boxplot,we can observe that median number of votes for both categories vary.
#### Restaurants accepting online orders get more votes from customers as there is a rating option poping up after each order through zomato application.

#### Is there any difference b/w price of restaurants accepting and not accepting online orders?

In [ ]:
sns.boxplot(x='online_order',y='approx_cost(for two people)',data=df)

In [ ]:
fig = px.box(df,x='online_order',y='approx_cost(for two people)')
fig.show()

#### Restaurants accepting online orders are more affordable than Restaurants who are accepting online orders

### Cheapest Rate for 2 people

In [ ]:
df['approx_cost(for two people)'].min()

### Most costly for 2 people

In [ ]:
df['approx_cost(for two people)'].max()

In [ ]:
df[df['approx_cost(for two people)']==6000]

In [ ]:
df[df['approx_cost(for two people)']==6000]['name']

##### distribution of cost for 2 people

In [ ]:
plt.figure(figsize=(6,6))
sns.distplot(df['approx_cost(for two people)'])
plt.show()

In [ ]:
px.histogram(df, x="approx_cost(for two people)")

##### most of the price lies between in a range of under 1000,it means most are affordable & very few are luxurious

### Most costly Rate for 2 people is served at which Restaurant what exactly is the dish involved in this and liked dish of that restaurant

In [ ]:
df[df['approx_cost(for two people)']==6000].loc[:,('name','cuisines','dish_liked')]

In [ ]:
df[df['approx_cost(for two people)']==6000][['name','cuisines','dish_liked']]

In [ ]:
data=df.copy()

In [ ]:
data.dtypes

In [ ]:
data.set_index('name',inplace=True)

### Top 10 Most Expensive restaurant with approx cost for 2 people

In [ ]:
data['approx_cost(for two people)'].nlargest(10).plot.bar()

In [ ]:
trace1 = go.Bar( 
        x = data['approx_cost(for two people)'].nlargest(10).index,
        y = data['approx_cost(for two people)'].nlargest(10),
        name= 'Priority')
iplot([trace1])

### Top 10 Cheapest restaurant with approx cost for 2 people

In [ ]:
data['approx_cost(for two people)'].nsmallest(10).plot.bar()

In [ ]:
trace1 = go.Bar( 
        x = data['approx_cost(for two people)'].nsmallest(10).index,
        y = data['approx_cost(for two people)'].nsmallest(10),
        name= 'Priority')
iplot([trace1])

### Top 10 Cheapest restaurant location wise with approx cost for 2 people 

In [ ]:
data.set_index('location',inplace=True)
data['approx_cost(for two people)'].nsmallest(10)

### all the restautant that are below than 500(budget hotel)

In [ ]:
data[data['approx_cost(for two people)']<=500]

In [ ]:
df_budget=data[data['approx_cost(for two people)']<=500].loc[:,('approx_cost(for two people)')]
df_budget=df_budget.reset_index()
df_budget.head()

In [ ]:
df_budget['approx_cost(for two people)'].value_counts().plot.bar()

### we can conclude that 300 and 400 cost dishes are maximum in count

In [ ]:
trace1 = go.Bar( 
        x = df_budget['approx_cost(for two people)'].value_counts().index,
        y = df_budget['approx_cost(for two people)'].value_counts(),
        name= 'Priority')
iplot([trace1])

### Restaurants that have better rating >4 and that are under budget too

In [ ]:
df[(df['rate']>=4) & (df['approx_cost(for two people)']<=500)].shape


### Total no. of Restaurants that have better rating >4 and that are under budget too ie less than 500

In [ ]:
df_new=df[(df['rate']>=4) & (df['approx_cost(for two people)']<=500)]
len(df_new['name'].unique())

### Total such various  affordable hotels at different location

In [ ]:
location=[]
total=[]
for loc,location_df in df_new.groupby('location'):
    location.append(loc)
    total.append(len(location_df['name'].unique()))
    
    

In [ ]:
len(location)

In [ ]:
len(total)

In [ ]:
location_df=pd.DataFrame(zip(location,total))
location_df.columns=['location','restaurant']
location_df.set_index('location',inplace=True)


In [ ]:
location_df.head(20)

In [ ]:
type(location)

In [ ]:

location_df['restaurant'].nlargest(10).plot.bar()
plt.gcf().autofmt_xdate()
plt.ylabel('Total restaurants')

In [ ]:
trace1 = go.Bar( 
        x = location_df['restaurant'].nlargest(10).index,
        y = location_df['restaurant'].nlargest(10),
        name= 'Priority')
iplot([trace1])

# To Visualise what are the names of those hotels

In [ ]:
location=[]
total=[]
for loc,location_df in df_new.groupby('location'):
    location.append(loc)
    total.append(location_df['name'].unique())
    

In [ ]:
afford=pd.DataFrame(zip(location,total))
afford.columns=['location','res_names']
afford.set_index('location',inplace=True)
afford.head()

### Finding Best budget Restaurants in any location

###### we will pass location and restaurant type as parameteres,function will return name of restaurants.

In [ ]:
def return_budget(location,restaurant):
    budget=df[(df['approx_cost(for two people)']<=400) & (df['location']==location) & 
                     (df['rate']>4) & (df['rest_type']==restaurant)]
    return(budget['name'].unique())

In [ ]:
return_budget('BTM',"Quick Bites")

### Which are the foodie areas?

In [ ]:
plt.figure(figsize=(10,7))
Restaurant_locations=df['location'].value_counts()[:20]
sns.barplot(Restaurant_locations,Restaurant_locations.index)

In [ ]:
Restaurant_locations=df['location'].value_counts()[:20]
trace1 = go.Bar( 
        x = Restaurant_locations.index,
        y = Restaurant_locations,
        name= 'Priority')
iplot([trace1])

#### We can see that BTM,HSR and Koranmangala 5th block has the most number of restaurants.
#### BTM dominates the section by having more than 5000 restaurants.

### geographical analysis

#### I need Latitudes & longitudes for each of the place for geaographical Data analysis,so to fetch lat,lon of each place,use Geopy

In [ ]:
df.shape

In [ ]:
len(df['location'].unique())

In [ ]:
locations=pd.DataFrame({"Name":df['location'].unique()})

In [ ]:
locations['new_Name']='Bangalore '+locations['Name']

In [ ]:
locations.head()

In [ ]:
!pip install geopy

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
lat_lon=[]
geolocator=Nominatim(user_agent="app")
for location in locations['Name']:
    location = geolocator.geocode(location)
    if location is None:
        lat_lon.append(np.nan)
    else:    
        geo=(location.latitude,location.longitude)
        lat_lon.append(geo)

In [ ]:
locations['geo_loc']=lat_lon

In [ ]:
locations.head()

In [ ]:
locations.to_csv('zomato_locations.csv',index=False)

##### We have found out latitude and longitude of each location listed in the dataset using geopy.
##### This is used to plot maps.

In [ ]:
Rest_locations=pd.DataFrame(df['location'].value_counts().reset_index())

In [ ]:
Rest_locations.columns=['Name','count']
Rest_locations.head()

##### now combine both the dataframes

In [ ]:
locations.head()

In [ ]:
locations.shape

In [ ]:
Rest_locations.shape

In [ ]:
Restaurant_locations=Rest_locations.merge(locations,on='Name',how="left").dropna()
Restaurant_locations.head()

In [ ]:
Restaurant_locations.shape

In [ ]:
Restaurant_locations['count'].max()

In [ ]:
type(Restaurant_locations['geo_loc'][0])

In [ ]:
def generateBaseMap(default_location=[12.97, 77.59], default_zoom_start=12):
    base_map = folium.Map(location=default_location, zoom_start=default_zoom_start)
    return base_map

In [ ]:
len(Restaurant_locations['geo_loc'])

In [ ]:
Restaurant_locations.isna().sum()

In [ ]:
Restaurant_locations['geo_loc'][0][0]

In [ ]:
Restaurant_locations['geo_loc'][0][1]

In [ ]:
np.array(Restaurant_locations['geo_loc'])

In [ ]:
#### unzip it
lat,lon=zip(*np.array(Restaurant_locations['geo_loc']))

In [ ]:
type(lat)

In [ ]:
Restaurant_locations['lat']=lat
Restaurant_locations['lon']=lon

In [ ]:
Restaurant_locations.head()

In [ ]:
!pip install folium

In [ ]:
import folium
from folium.plugins import HeatMap
basemap=generateBaseMap()

In [ ]:
basemap

In [ ]:
Restaurant_locations[['lat','lon','count']].values.tolist()

In [ ]:
HeatMap(Restaurant_locations[['lat','lon','count']].values.tolist(),zoom=20,radius=15).add_to(basemap)

In [ ]:
basemap

##### It is clear that restaurants tend to concentrate in central bangalore area.
##### The clutter of restaurants lowers are we move away from central.
##### So,potential restaurant entrepreneurs can refer this and find out good locations for their venture.

#### note heatmap is good when we have latitude,longitude or imporatnce of that particular place or count of that place

### Heatmap of North Indian restaurants

In [ ]:
df.head()

In [ ]:
df2= df[df['cuisines']=='North Indian']
df2.head()

In [ ]:
north_india=df2.groupby(['location'],as_index=False)['url'].agg('count')
north_india.columns=['Name','count']
north_india.head()

In [ ]:
north_india=north_india.merge(locations,on="Name",how='left').dropna()

In [ ]:
north_india.head()

In [ ]:
north_india['lan'],north_india['lon']=zip(*north_india['geo_loc'].values)


In [ ]:
north_india.drop(['geo_loc'],axis=1)

In [ ]:
basemap=generateBaseMap()
HeatMap(north_india[['lan','lon','count']].values.tolist(),zoom=20,radius=15).add_to(basemap)
basemap

###### What about South Indian cuisines?

In [ ]:
df3= df[df['cuisines']=='South Indian']
south_india=df2.groupby(['location'],as_index=False)['url'].agg('count')
south_india.columns=['Name','count']
south_india=south_india.merge(locations,on="Name",how='left').dropna()
south_india['lan'],south_india['lon']=zip(*south_india['geo_loc'].values)


In [ ]:
south_india=south_india.drop(['geo_loc'],axis=1)

In [ ]:
south_india.head()

In [ ]:
basemap=generateBaseMap()
HeatMap(south_india[['lan','lon','count']].values.tolist(),zoom=20,radius=15).add_to(basemap)
basemap

##### Which are the most popular casual dining restaurant chains?

In [ ]:
df_1=df.groupby(['rest_type','name']).agg('count')
datas=df_1.sort_values(['url'],ascending=False).groupby(['rest_type'],
                as_index=False).apply(lambda x : x.sort_values(by="url",ascending=False).head(3))['url'].reset_index().rename(columns={'url':'count'})

In [ ]:
datas

In [ ]:
df_1=df.groupby(['rest_type','name']).agg('count')
df_1

In [ ]:
df_1.sort_values(['url'],ascending=False)

In [ ]:
df_1.sort_values(['url'],ascending=False).groupby(['rest_type'],as_index=False).apply(lambda x : x.sort_values(by="url",ascending=False))

In [ ]:
df_1.sort_values(['url'],ascending=False).groupby(['rest_type'],as_index=False).apply(lambda x : x.sort_values(by="url",ascending=False))['url']

In [ ]:
df_1.sort_values(['url'],ascending=False).groupby(['rest_type'],as_index=False).apply(lambda x : x.sort_values(by="url",ascending=False))['url'].reset_index()

In [ ]:
dataset=df_1.sort_values(['url'],ascending=False).groupby(['rest_type'],
                as_index=False).apply(lambda x : x.sort_values(by="url",ascending=False))['url'].reset_index().rename(columns={'url':'count'})

In [ ]:
dataset

##### Which are the most popular casual dining restaurant chains?

In [ ]:
casual=dataset[dataset['rest_type']=='Casual Dining']
casual

In [ ]:
df.shape

We can see tht Empire restaurant,Beijing bites and Mani's dum biriyani are the most popular casual dining restaurant chains in Bangalore.
We will inspect them further...

##### Where are their outlets located?

#### Wordcloud of dishes liked by cuisines.

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
df.head()

In [ ]:
df['update_dish_liked']=df['dish_liked'].apply(lambda x : x.split(',') if type(x)==str else [''])

In [ ]:
df.head()

In [ ]:
df['rest_type'].value_counts()[:9].index

In [ ]:
rest=df['rest_type'].value_counts()[:9].index

##### create wordcloud for each and every restaurant

In [ ]:
from wordcloud import WordCloud, STOPWORDS 

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(axis='index',subset=['rest_type'],inplace=True)

In [ ]:
df.dropna(axis='index',subset=['dish_liked'],inplace=True)

In [ ]:
df.isna().sum()

##### wordcloud for 1 restaurant

In [ ]:
data=df[df['rest_type']=='Quick Bites']

In [ ]:
data['dish_liked']

In [ ]:
stopwords=set(STOPWORDS)

In [ ]:
dishes=''
for word in data['dish_liked']:
    words=word.split()
    # Converts each token into lowercase 
    for i in range(len(words)): 
        words[i] = words[i].lower() 
    dishes=dishes+ " ".join(words)+" "
wordcloud = WordCloud(max_font_size=None, background_color='white', collocations=False,stopwords = stopwords,width=1500, height=1500).generate(dishes)
plt.imshow(wordcloud)
plt.axis("off")

In [ ]:
def produce_wordcloud(rest):
    
    plt.figure(figsize=(20,30))
    for i,restaurant in enumerate(rest):
        plt.subplot(3,3,i+1)
        dishes=''
        data=df[df['rest_type']==restaurant]
        for word in data['dish_liked']:
            words=word.split()
            # Converts each token into lowercase 
            for i in range(len(words)): 
                words[i] = words[i].lower() 
            dishes=dishes+ " ".join(words)+" "
        wordcloud = WordCloud(max_font_size=None, background_color='white', collocations=False,stopwords = stopwords,width=1500, height=1500).generate(dishes)
        plt.imshow(wordcloud)
        plt.title(restaurant)
        plt.axis("off")

In [ ]:
stopwords = set(STOPWORDS) 
produce_wordcloud(rest)

##### analysing Reviews of Particular Restaurant

In [ ]:
df.head()

In [ ]:
df['reviews_list'][0]

In [ ]:
data=df['reviews_list'][0].lower()
data

In [ ]:
import re
data2=re.sub('[^a-zA-Z]', ' ',data)
data2

In [ ]:
data3=re.sub('rated', ' ',data2)
data3

In [ ]:
data4=re.sub('x',' ',data3)
data4

##### remove multiple spaces

In [ ]:
re.sub(' +',' ',data4)

#### analyse review of a particular restaurant

In [ ]:
dataset=df[df['rest_type']=='Quick Bites']

In [ ]:
type(dataset['reviews_list'][3])

In [ ]:
total_review=' '
for review in dataset['reviews_list']:
    review=review.lower()
    review=re.sub('[^a-zA-Z]', ' ',review)
    review=re.sub('rated', ' ',review)
    review=re.sub('x',' ',review)
    review=re.sub(' +',' ',review)
    total_review=total_review + str(review)
 

In [ ]:
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(total_review) 
# plot the WordCloud image                        
plt.figure(figsize = (8, 8)) 
plt.imshow(wordcloud) 
plt.axis("off") 

In [ ]:
def importance(restaurant):
    dataset=df[df['rest_type']==restaurant]
    total_review=' '
    for review in dataset['reviews_list']:
        review=review.lower()
        review=re.sub('[^a-zA-Z]', ' ',review)
        review=re.sub('rated', ' ',review)
        review=re.sub('x',' ',review)
        review=re.sub(' +',' ',review)
        total_review=total_review + str(review)
    wordcloud = WordCloud(width = 800, height = 800, 
            background_color ='white', 
            stopwords = set(STOPWORDS), 
            min_font_size = 10).generate(total_review) 
    # plot the WordCloud image                        
    plt.figure(figsize = (8, 8)) 
    plt.imshow(wordcloud) 
    plt.axis("off") 

In [ ]:
importance('Quick Bites')